In [1]:
import pandas as pd
import requests
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
import pymongo
import json
from config import apikey
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import pymongo

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
full=pd.read_csv('../01_Scrape/full.csv')
full.head()

,common_name,scientific_name,url,category,depth,distribution,images,size
0,Moorish Idol,Zanclus cornutus,https://reefguide.org/moorishidol.html,Large Oval Fish,10-300 ft. (3-90 m),"Indo-Pacific, Hawaii, Gulf of California, Paci...",['https://reefguide.org/pix/thumb2/moorishidol...,5 to 8 in. (13 to 20 cm)
1,Longsnout Butterflyfish,Prognathodes aculeatus,https://reefguide.org/longsnoutbutter.html,Large Oval Fish,30-200 ft. (10-60 m),"Caribbean, Bahamas, Florida, Bermuda, Gulf of ...",['https://reefguide.org/pix/thumb2/longsnoutbu...,2 to 3 in. (5 to 8 cm)
2,Orange-Banded Coralfish,Coradion chrysozonus,https://reefguide.org/coradionchrysozonus.html,Large Oval Fish,10-180 ft. (3-55 m),"Indo-West Pacific, Great Barrier Reef",['https://reefguide.org/pix/thumb2/coradionchr...,Up to 6 in. (15 cm)
3,Two-Eyed Coralfish,Coradion melanopus,https://reefguide.org/coradionmelanopus.html,Large Oval Fish,30-100 ft. (10-30 m),Asian Pacific,['https://reefguide.org/pix/coradionmelanopus1...,Up to 6 in. (15 cm)
4,Humphead Bannerfish,Heniochus varius,https://reefguide.org/humpheadbannerfish.html,Large Oval Fish,6-100 ft. (2-30 m),"West Pacific, Indonesia, French Polynesia",['https://reefguide.org/pix/thumb2/humpheadban...,5 to 8 in. (13 to 20 cm)


In [3]:
api_key='?token='+apikey
full=full[['common_name', 'scientific_name', 'category', 'url', 'depth', 'distribution', 'size']]

# Countries

In [4]:
url_country='http://apiv3.iucnredlist.org/api/v3/species/countries/name/'
country_dic={}
for i, species in tqdm(full.iterrows()):
    combined_url=url_country+species['scientific_name']+api_key
    pre_df=[]
    try:
        response=requests.get(combined_url).json()
        response['category']=species['category']
        pre_df.append(response)
        common_name=species['common_name']
        country_dic[f"{common_name}"]=pre_df
    except:
        pass


In [7]:
#loading to pymongo (so I don't have to make calls again to API)
#Create Connection to MongoDB
conn="mongodb://localhost:27017"
client=pymongo.MongoClient(conn)

MarineBuddy = client["MarineBuddy"]

#create an details_v2 collection into collection
Countries=MarineBuddy['country_occurence']
Countries.update({}, country_dic, upsert=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  # Remove the CWD from sys.path while we load stuff.


{'n': 1,
 'nModified': 0,
 'upserted': ObjectId('5cabf738fe2eaaeab60990d6'),
 'ok': 1.0,
 'updatedExisting': False}

In [28]:
#parsing through country_dic to create dictionary with array of countries specific to species
pre_df_country=[]
for k,v in country_dic.items():

    pre_df={}
    try:
        common_name=k
        scientific_name=v[0]['name']
        category=v[0]['category']
        country_list=v[0]['result']
        countries=[]
        for item in country_list:
            country=item['country']
            countries.append(country)
        pre_df['common_name']=common_name
        pre_df['scientific_name']=scientific_name
        pre_df['category']=category
        pre_df['countries']=countries
    
    except:
        pass
    
    pre_df_country.append(pre_df)

In [29]:
#turn pre_df_country to dataframe (contains array of countries in cells)
df=pd.DataFrame(pre_df_country)
df=df.dropna(axis=0)

In [30]:
#explode array of countries into multiple rows for SQL
rows = []
_ = df.apply(lambda row: [rows.append([row['category'], row['common_name'], nn, row['scientific_name']]) 
                         for nn in row.countries], axis=1)
new_df= pd.DataFrame(rows, columns=df.columns)

In [39]:
#open geojson with countries
country_json=[]
with open("C:/Users/Sara's PC/Documents/Coding/FinalProject/API_Calls/countries.json") as f:
    data = json.load(f)
    
    
for item in data['features']:
    country_json.append(item['properties']['name'])

In [40]:
##making sure country names match between geojson and API calls (with fuzzy match)
unique_countries=new_df['countries'].unique()

temp_list=[]
for country in unique_countries:
    try:
        result, score = process.extractOne(country, country_json,scorer=fuzz.partial_ratio) #, score_cutoff=100)
        temp_list.append({'fish_country':country, 'country_json': result, 'score':score})
    except:
        pass
countries_fixed = pd.DataFrame(temp_list)

In [41]:
#isolate fuzzy scores less than 100
countries_fixed[countries_fixed['score']<100]

#correct for certain countries
fix_countries_dict={'Cocos (Keeling) Islands': 'Costa Rica',
                    'Korea, Republic of': 'South Korea',
                    'Norfolk Island': 'United Kingdom',
                    'Mauritius': 'Madagascar',
                    'Maldives': 'Sri Lanka',
                    'French Polynesia': 'Fiji',
                    'Tanzania, United Republic of': 'United Republic of Tanzania',
                    'Viet Nam': 'Vietnam',
                    "Côte d'Ivoire":'Ivory Coast',
                    'Singapore': 'Malaysia',
                    'Gibraltar': 'Spain',
                    'Palestine, State of':'West Bank',
                    'Hong Kong': 'China',
                    "Lao People's Democratic Republic": 'China',
                    'Congo, The Democratic Republic of the': 'Democratic Republic of the Congo',
                    'Faroe Islands': 'Denmark',
                    'Monaco': 'France',
                    'United States': 'United States of America',
                    'Taiwan, Province of China': 'China',
                    "Korea, Democratic People's Republic of": 'North Korea',
                    'Venezuela, Bolivarian Republic of': "Venezuela",
                    'Serbia': "Republic of Serbia"}

#replacing countries based on the fix_countries_dic 
for k,v in fix_countries_dict.items():
    new_df['countries'].loc[new_df['countries']==k] = v


In [88]:
#finding countries that are in new_df table but not in countries geojson
countries_to_remove=[]
for i in new_df['countries'].values:
    if i in country_json:
        pass
    else:
        countries_to_remove.append(i)

countries_to_remove=list(set(countries_to_remove))

In [43]:
#remove the countries that are not in geojson from new_df
country_occurence_table=new_df[~new_df['countries'].isin(countries_to_remove)]
country_occurence_table.head()

country_occurence_table.drop_duplicates(subset=['category', 'common_name', 'countries', 'scientific_name'], keep='first', inplace=True)
country_occurence_table.reset_index()
country_occurence_table.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


(20746, 4)

In [44]:
country_occurence_table.to_csv('../03_Dump_Files/country_occurence.csv', index=False)
country_occurence_table.head()

,category,common_name,countries,scientific_name
1,Large Oval Fish,Moorish Idol,Australia,Zanclus cornutus
3,Large Oval Fish,Moorish Idol,Costa Rica,Zanclus cornutus
4,Large Oval Fish,Moorish Idol,Chile,Zanclus cornutus
5,Large Oval Fish,Moorish Idol,China,Zanclus cornutus
6,Large Oval Fish,Moorish Idol,Colombia,Zanclus cornutus


In [45]:
#country table for SQL
unique_countries=country_occurence_table['countries'].unique()
unique_countries_df=pd.DataFrame(unique_countries, columns=['country'])
unique_countries_df['country_id']=unique_countries_df.index+1
unique_countries_df.to_csv('../Dump_Files/countries.csv', index=False)

In [55]:
#get unique species that are in country_occurence database to run other API calls
species_final=country_occurence_table[['category', 'common_name', 'scientific_name']]
species_final.drop_duplicates(subset=['category', 'common_name', 'scientific_name'], keep='first', inplace=True)
species_final.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(972, 3)

In [56]:
species_final=pd.merge(species_final, full, on=['common_name', 'scientific_name', 'category'], how='left')
species_final.head()

,category,common_name,scientific_name,url,depth,distribution,size
0,Large Oval Fish,Moorish Idol,Zanclus cornutus,https://reefguide.org/moorishidol.html,10-300 ft. (3-90 m),"Indo-Pacific, Hawaii, Gulf of California, Paci...",5 to 8 in. (13 to 20 cm)
1,Large Oval Fish,Longsnout Butterflyfish,Prognathodes aculeatus,https://reefguide.org/longsnoutbutter.html,30-200 ft. (10-60 m),"Caribbean, Bahamas, Florida, Bermuda, Gulf of ...",2 to 3 in. (5 to 8 cm)
2,Large Oval Fish,Orange-Banded Coralfish,Coradion chrysozonus,https://reefguide.org/coradionchrysozonus.html,10-180 ft. (3-55 m),"Indo-West Pacific, Great Barrier Reef",Up to 6 in. (15 cm)
3,Large Oval Fish,Two-Eyed Coralfish,Coradion melanopus,https://reefguide.org/coradionmelanopus.html,30-100 ft. (10-30 m),Asian Pacific,Up to 6 in. (15 cm)
4,Large Oval Fish,Humphead Bannerfish,Heniochus varius,https://reefguide.org/humpheadbannerfish.html,6-100 ft. (2-30 m),"West Pacific, Indonesia, French Polynesia",5 to 8 in. (13 to 20 cm)


# Details

In [ ]:
# #loading from pymongo
# #Create Connection to MongoDB
# conn="mongodb://localhost:27017"
# client=pymongo.MongoClient(conn)


# MarineBuddy = client["MarineBuddy"]

# details_dic_mongo = MarineBuddy.details.find()
# details_dic=[]
# for result in details_dic_mongo:
#     details_dic.append(result)

In [57]:
#Make API calls with final 972 species
url_1='http://apiv3.iucnredlist.org/api/v3/species/'
url_2='http://apiv3.iucnredlist.org/api/v3/species/history/name/'
url_3='http://apiv3.iucnredlist.org/api/v3/threats/species/name/'
url_4='http://apiv3.iucnredlist.org/api/v3/habitats/species/name/'
url_5='http://apiv3.iucnredlist.org/api/v3/weblink/'
url_6='http://apiv3.iucnredlist.org/api/v3/measures/species/name/'

details_dic={}
for i, species in tqdm(species_final.iterrows()):
    df={}
    common_name=species['common_name']
    #citation (1)
    combined_url=url_1+species['scientific_name']+api_key
    try:
        response=requests.get(combined_url).json()
        depth_lower=response['result'][0]['depth_lower']
        depth_upper=response['result'][0]['depth_upper']
        family=response['result'][0]['family']
        genus=response['result'][0]['genus']
        kingdom=response['result'][0]['kingdom']
        order=response['result'][0]['order']
        phylum=response['result'][0]['phylum']
        population_trend=response['result'][0]['population_trend']
        status=response['result'][0]['category']
    except:
        pass
    
    df['scientific_name']=species['scientific_name']
    df['category']=species['category']
    df['url']=species['url']
    df['depth']=species['depth']
    df['distribution']=species['distribution']
    df['size']=species['size']
    df['depth lower']=depth_lower
    df['depth upper']=depth_upper
    df['family']=family
    df['genus']=genus
    df['kingdom']=kingdom
    df['order']=order
    df['phylum']=phylum
    df['population trend']=population_trend
    df['status']=status
    
    #history(2)
    combined_url=url_2+species['scientific_name']+api_key
    try:
        response=requests.get(combined_url).json()
        trend=response['result']
    except:
        pass
    
    df['historic trend']=trend
    
    #threats(3)
    combined_url=url_3+species['scientific_name']+api_key
    try:
        response=requests.get(combined_url).json()
        threats_full=response['result']
        threats=set()
        for threat in threats_full:
            threats.add(threat['title'])
        threats=list(threats)
    except:
        pass
    
    df['threats']=threats
    
    #habitats(4)
    combined_url=url_4+species['scientific_name']+api_key
    try:
        response=requests.get(combined_url).json()
        habitats_full=response['result']
        habitats=set()
        for item in habitats_full:
            habitats.add(item['habitat'])
        habitats=list(habitats)
    except:
        pass
    
    df['habitats']=habitats
      
    #weblink(5)
    combined_url=url_5+species['scientific_name']+api_key
    try:
        response=requests.get(combined_url).json()
        IUCN_weblink=response['rlurl']
    except:
        pass
    
    df['IUCN weblink']=IUCN_weblink
    
    #measures(6)
    combined_url=url_6+species['scientific_name']+api_key
    try:
        response=requests.get(combined_url).json()
        measures_full=response['result']
        measures=set()
        for item in measures_full:
            measures.add(item['title'])
        measures=list(measures)
    except:
        pass
    
    df['measures']=measures
    
    details_dic[f"{common_name}"]=df

In [58]:
#loading to pymongo (to avoid future API calls)
#Create Connection to MongoDB
conn="mongodb://localhost:27017"
client=pymongo.MongoClient(conn)

MarineBuddy = client["MarineBuddy"]

#create an details_v2 collection into collection
Details=MarineBuddy['details']
Details.update({}, details_dic, upsert=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  # Remove the CWD from sys.path while we load stuff.


{'n': 1,
 'nModified': 0,
 'upserted': ObjectId('5cac0ecdfe2eaaeab609934a'),
 'ok': 1.0,
 'updatedExisting': False}

In [59]:
#parsing through details_dic
pre_df_details=[]
for k,v in details_dic.items():
    pre_df={}
    try:
        common_name=k
        scientific_name=v['scientific_name']
        category=v['category']
        IUCN_weblink=v['IUCN weblink']
        depth=v['depth']
        depth_lower=v['depth lower']
        depth_upper=v['depth upper']
        distribution=v['distribution']
        family=v['family']
        genus=v['genus']
        order=v['order']
        phylum=v['phylum']
        population_trend=v['population trend']
        size=v['size']
        url=v['url']
        status=v['status']
        
        pre_df['common_name']=common_name
        pre_df['scientific_name']=scientific_name
        pre_df['category']=category
        pre_df['IUCN_weblink']=IUCN_weblink
        pre_df['depth']=depth
        pre_df['depth_lower']=depth_lower
        pre_df['depth_upper']=depth_upper
        pre_df['distribution']=distribution
        pre_df['family']=family
        pre_df['genus']=genus
        pre_df['order']=order
        pre_df['phylum']=phylum
        pre_df['population_trend']=population_trend
        pre_df['size']=size
        pre_df['url']=url
        pre_df['status']=status
        
    except:
        pass
    
    pre_df_details.append(pre_df)

In [60]:
details_df=pd.DataFrame(pre_df_details)
details_df=details_df.reset_index(drop=True)
details_df.head()

,IUCN_weblink,category,common_name,depth,depth_lower,depth_upper,distribution,family,genus,order,phylum,population_trend,scientific_name,size,status,url
0,https://www.iucnredlist.org/species/69741115/6...,Large Oval Fish,Moorish Idol,10-300 ft. (3-90 m),180.0,1.0,"Indo-Pacific, Hawaii, Gulf of California, Paci...",ZANCLIDAE,Zanclus,PERCIFORMES,CHORDATA,Stable,Zanclus cornutus,5 to 8 in. (13 to 20 cm),LC,https://reefguide.org/moorishidol.html
1,https://www.iucnredlist.org/species/165627/607...,Large Oval Fish,Longsnout Butterflyfish,30-200 ft. (10-60 m),100.0,1.0,"Caribbean, Bahamas, Florida, Bermuda, Gulf of ...",CHAETODONTIDAE,Prognathodes,PERCIFORMES,CHORDATA,Stable,Prognathodes aculeatus,2 to 3 in. (5 to 8 cm),LC,https://reefguide.org/longsnoutbutter.html
2,https://www.iucnredlist.org/species/165702/609...,Large Oval Fish,Orange-Banded Coralfish,10-180 ft. (3-55 m),60.0,3.0,"Indo-West Pacific, Great Barrier Reef",CHAETODONTIDAE,Coradion,PERCIFORMES,CHORDATA,Stable,Coradion chrysozonus,Up to 6 in. (15 cm),LC,https://reefguide.org/coradionchrysozonus.html
3,https://www.iucnredlist.org/species/155193/474...,Large Oval Fish,Two-Eyed Coralfish,30-100 ft. (10-30 m),50.0,2.0,Asian Pacific,CHAETODONTIDAE,Coradion,PERCIFORMES,CHORDATA,Stable,Coradion melanopus,Up to 6 in. (15 cm),LC,https://reefguide.org/coradionmelanopus.html
4,https://www.iucnredlist.org/species/165652/608...,Large Oval Fish,Humphead Bannerfish,6-100 ft. (2-30 m),30.0,2.0,"West Pacific, Indonesia, French Polynesia",CHAETODONTIDAE,Heniochus,PERCIFORMES,CHORDATA,Stable,Heniochus varius,5 to 8 in. (13 to 20 cm),LC,https://reefguide.org/humpheadbannerfish.html


In [61]:
#replace all Nan for depth_upper and depth_lower with 0
details_df['depth_upper'].fillna(0, inplace=True)
details_df['depth_lower'].fillna(0, inplace=True)
details_df['depth_upper']=details_df['depth_upper'].round(0).astype(int)
details_df['depth_lower']=details_df['depth_lower'].round(0).astype(int)

details_df.shape

(972, 16)

In [62]:
#if depth column is blank, take depth_upper to depth_lower as depth range 
# unless there are no values for depth_upper and depth_lower (equal to 0), in which case set depth to 'Unknown'
for i, row in details_df.iterrows():
    if details_df.loc[i,'depth']==' ':
        if details_df.loc[i, 'depth_lower']!=0:
            details_df.loc[i, 'depth']="{}-{} m".format(details_df.loc[i, 'depth_upper'], details_df.loc[i, 'depth_lower'])
        else:
            details_df.loc[i, 'depth']='Unknown'

In [63]:
details_df=details_df[['IUCN_weblink',
                       'category',
                       'common_name',
                       'depth',
                       'distribution',
                       'family',
                       'genus',
                       'order',
                       'phylum',
                       'population_trend',
                       'scientific_name',
                       'url',
                       'status']]

In [64]:
details_df.shape

(972, 13)

In [65]:
details_df.to_csv('../03_Dump_Files/details.csv', index=False)

In [66]:
#measures table
pre_df_measures=[]
for k,v in details_dic.items():
    pre_df={}
    try:
        common_name=k
        scientific_name=v['scientific_name']
        category=v['category']
        measures_list=v['measures']
        measures=[]
        for item in measures_list:
            measure=item
            measures.append(measure)
        pre_df['common_name']=common_name
        pre_df['scientific_name']=scientific_name
        pre_df['category']=category
        pre_df['measures']=measures
    
    except:
        pass
    
    pre_df_measures.append(pre_df)

In [68]:
measures_df=pd.DataFrame(pre_df_measures)
measures_df.head()

,category,common_name,measures,scientific_name
0,Large Oval Fish,Moorish Idol,[],Zanclus cornutus
1,Large Oval Fish,Longsnout Butterflyfish,[],Prognathodes aculeatus
2,Large Oval Fish,Orange-Banded Coralfish,[],Coradion chrysozonus
3,Large Oval Fish,Two-Eyed Coralfish,[],Coradion melanopus
4,Large Oval Fish,Humphead Bannerfish,[],Heniochus varius
5,Large Oval Fish,Pennant Bannerfish,[],Heniochus chrysostomus
6,Large Oval Fish,Masked Bannerfish,[],Heniochus monoceros
7,Large Oval Fish,Singular Bannerfish,[],Heniochus singularius
8,Large Oval Fish,Schooling Bannerfish,[],Heniochus diphreutes
9,Large Oval Fish,Longnose Butterflyfish,[],Forcipiger flavissimus


In [69]:
#explode array of measures into multiple rows
rows = []
_ = measures_df.apply(lambda row: [rows.append([row['category'], row['common_name'], nn, row['scientific_name']]) 
                         for nn in row.measures], axis=1)
measures_df = pd.DataFrame(rows, columns=measures_df.columns)


measures_df=measures_df.dropna(axis=0)

In [73]:
measures_df.to_csv('../03_Dump_Files/measures.csv', index=False)
measures_df.shape

(1159, 4)

In [74]:
#threats table
pre_df_threats=[]
for k,v in details_dic.items():
    pre_df={}
    try:
        common_name=k
        scientific_name=v['scientific_name']
        category=v['category']
        threats_list=v['threats']
        threats=[]
        for item in threats_list:
            threat=item
            threats.append(threat)
        pre_df['common_name']=common_name
        pre_df['scientific_name']=scientific_name
        pre_df['category']=category
        pre_df['threats']=threats
    
    except:
        pass
    
    pre_df_threats.append(pre_df)

In [75]:
threats_df=pd.DataFrame(pre_df_threats)
threats_df.head()

,category,common_name,scientific_name,threats
0,Large Oval Fish,Moorish Idol,Zanclus cornutus,[]
1,Large Oval Fish,Longsnout Butterflyfish,Prognathodes aculeatus,"[Habitat shifting & alteration, Intentional us..."
2,Large Oval Fish,Orange-Banded Coralfish,Coradion chrysozonus,[Intentional use: (subsistence/small scale) [h...
3,Large Oval Fish,Two-Eyed Coralfish,Coradion melanopus,[Intentional use: (subsistence/small scale) [h...
4,Large Oval Fish,Humphead Bannerfish,Heniochus varius,[Intentional use: (subsistence/small scale) [h...


In [76]:
#explode array of threats into multiple rows
rows = []
_ = threats_df.apply(lambda row: [rows.append([row['category'], row['common_name'], row['scientific_name'],  nn]) 
                         for nn in row.threats], axis=1)
threats_df = pd.DataFrame(rows, columns=threats_df.columns)


threats_df=threats_df.dropna(axis=0)
threats_df.shape

(3890, 4)

In [79]:
threats_df.to_csv('../03_Dump_Files/threats.csv', index=False)

In [80]:
#historic trend table
pre_df_trend=[]
for k,v in details_dic.items():
    pre_df={}
    try:
        common_name=k
        scientific_name=v['scientific_name']
        category=v['category']
        trend_list=v['historic trend']
        statuses=[]
        for item in trend_list:
            year=item['year']
            code=item['category']
            statuses.append((year, code))
        pre_df['common_name']=common_name
        pre_df['scientific_name']=scientific_name
        pre_df['statuses']=statuses
    
    except:
        pass
    
    pre_df_trend.append(pre_df)

In [81]:
trends_df=pd.DataFrame(pre_df_trend)
trends_df.head()

,common_name,scientific_name,statuses
0,Moorish Idol,Zanclus cornutus,"[(2016, Least Concern)]"
1,Longsnout Butterflyfish,Prognathodes aculeatus,"[(2010, Least Concern)]"
2,Orange-Banded Coralfish,Coradion chrysozonus,"[(2010, Least Concern)]"
3,Two-Eyed Coralfish,Coradion melanopus,"[(2010, Least Concern)]"
4,Humphead Bannerfish,Heniochus varius,"[(2010, Least Concern)]"


In [82]:
#explode array of statuses (tuples) into multiple rows
rows = []
_ = trends_df.apply(lambda row: [rows.append([row['common_name'], row['scientific_name'], nn]) 
                         for nn in row.statuses], axis=1)
trends_df = pd.DataFrame(rows, columns=trends_df.columns)


threats_df=threats_df.dropna(axis=0)
trends_df.shape

(1604, 3)

In [83]:
trends_df[['year', 'status']] = pd.DataFrame(trends_df['statuses'].tolist(), index=trends_df.index)
trends_df=trends_df[['common_name', 'scientific_name', 'year', 'status']]

In [84]:
trends_df.to_csv('../03_Dump_Files/trends.csv', index=False)